In [ ]:
# conda env list

# conda create -n [이름] python[버전]  -> 버전 안쓰면 최신버전을
# conda activate [이름]

# conda deactivate
# conda remove -n[이름] --all

<절차>
1. 입력
2. 토큰화 및 시퀀스 변화
3. 패딩(길이 고정)
4. 임베딩 (단어 벡터화)
5. 1D Convolution + poling 반복
6. Flatten
7. Dense(은닉)
8. 출력(소프트맥스, 이진분류)
9. 학습(Adam, Binary_Crossentropy)
10. 검증/ 테스트 평가
11. 시각화

---

- 말뭉치 로딩 (nltk) : 데이터 로딩
- 토큰화(빈도 기반 인덱싱) : 텍스트를 숫자로 변환
- 시퀀스 패딩 : 고정길이 배치
- 임베딩 : 단어를 밀집기준 벡터화  
    - 임베딩
        - 한계 : 작은 데이터에서는 일반화 부족
        - 발전 : 사전학습(Word2vec), 문맥적 임베딩(BERT, GPT)

<span style="color: Gold"> 말뭉치 로딩 (nltk) : 데이터 로딩

<span style="color: Gold"> 1. 토큰화(빈도 기반 인덱싱) : 텍스트를 숫자로 변환

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# sample data
texts = [
    "I really love this movie",
    'I hate this boring film',
    'love love great film'
]

# 토큰화 객체 (최대 단어 10, oov 토큰 지정)
tokenizer = Tokenizer(num_words=10, oov_token = 'UNK') # 최대 사용 단어 num_words=10 가장 많이 나오는 것중에 10개만 보겟다/ oov_token 내가 학습할 때 본 적 없는 단어. 즉 사전에 없는 단어
tokenizer.fit_on_texts(texts)
print(f'단어인덱스 : {tokenizer.word_index}')       # 토큰화된 단어를 보는 것


단어인덱스 : {'UNK': 1, 'love': 2, 'i': 3, 'this': 4, 'film': 5, 'really': 6, 'movie': 7, 'hate': 8, 'boring': 9, 'great': 10}


<span style="color: Gold"> 2. 시퀀스 패딩 : 고정길이 배치

In [ ]:
seqs = tokenizer.texts_to_sequences(texts) # 위에 문장을 숫자로 바꾸는 것
print(f'원본 시퀀스 : {seqs}')

# 패딩 (최대 길이를 6)
padded = pad_sequences(seqs, maxlen=6, padding='post')       # padding='post' 뒤에 패딩 넣기/ 앞이면 'pre'
print(f' 패딩결과 : {padded} 사이즈 : {padded.shape}')

원본 시퀀스 : [[3, 6, 2, 4, 7], [3, 8, 4, 9, 5], [2, 2, 1, 5]]
 패딩결과 : [[3 6 2 4 7 0]
 [3 8 4 9 5 0]
 [2 2 1 5 0 0]] 사이즈 : (3, 6)


<span style="color: Gold"> 3. 임베딩 : 단어를 밀집기준 벡터화

In [ ]:
import tensorflow as tf
# 패딩된 시퀀스 padded
vocab_size = 11     # UNK를 포함하여 단어 인덱스 최대값 +1 -> num_words=10 +1을 해준다
embed_dim = 4 # 임베딩 차원. 벡터가 4차원
model = tf.keras.Sequential ([
    tf.keras.layers.Embedding(input_dim = vocab_size, output_dim=embed_dim, input_length = 6)
]) # input_dim-> 입력 가능한 단어 인덱스 최대값+1 / output_dim -> 각 단어가 바뀔 벡터 차원 수/  input_length  -> 입력 시퀀스(문장)의 길이를 지정
embeddings = model.predict(padded)
print(f'임베딩 텐서 모양 : {embeddings.shape}')
print(f'첫 문장 첫 단어 벡터 : {embeddings[0,0,:]}')

c:\Users\SAMSUNG\anaconda3\envs\deep\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
임베딩 텐서 모양 : (3, 6, 4)
첫 문장 첫 단어 벡터 : [-0.00802351  0.04552143 -0.04114251  0.02707568]


<span style="color: Gold"> 4. 1D Convolution

In [ ]:
import numpy as np
import tensorflow as tf
# 임의의 시퀀스 (배치 =1, 길이=6, 임베딩=4) → 1개의 문장, 6단어, 각 단어 4차원 벡터
x = np.random.randn(1,6,4).astype('float32')
conv = tf.keras.layers.Conv1D(
    filters = 2, # 2개의 패턴을 감지-> 긍정 부정을 감지
    kernel_size = 3, # 3-gram-> 3단어씩 묶어서(3-gram) 특징 추출
    activation = 'relu' # 음수값은 0으로, 양수만 통과
)
y = conv(x)
print(f'입력 shape {x.shape}')
print(f'출력 shape {y.shape}') # 4인 이유 -> 123,234,345,456
print(f'출력값 {y.numpy()}')

입력 shape (1, 6, 4)
출력 shape (1, 4, 2)
출력값 [[[0.         1.5551264 ]
  [1.1359663  0.96499085]
  [1.825316   0.        ]
  [0.         0.        ]]]


<span style="color: Gold"> 5. MaxPooling

In [22]:
pool = tf.keras.layers.MaxPool1D(pool_size =2)
pooled = pool(y)
print(f'폴링 전 : {y.shape}')
print(f'폴링 후 : {pooled.shape}')


폴링 전 : (1, 4, 2)
폴링 후 : (1, 2, 2)


---

<span style="color: pink"> 종합

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

texts = [
    "I really love this movie", # 긍정
    'I hate this boring film',  # 부정
    'great love movie',         # 긍정
    'boring hate film'          # 부정
]
labels = np.array([0,1,0,1])    # 긍정,부정,긍정,부정 라벨링

# 토큰화
tokenizer = Tokenizer(num_words=50,oov_token = 'UNK') 
tokenizer.fit_on_texts(texts)                       # 단어 사전 생성
seqs = tokenizer.texts_to_sequences(texts)          # 단어 사전 기반으로 단어들을 숫자로 변경

# 패딩 (최대 길이를 6)
x = pad_sequences(seqs, maxlen=6, padding='post')       # padding='post' 뒤에 패딩 넣기

# 임베딩
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = 51, output_dim = 8, input_length = 6),
    tf.keras.layers.Conv1D(filters = 16, kernel_size = 3, activation = 'relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')   # 마지막 출력층은 0,1 긍정부정으로 설정을 했으니까 2개로 해서 비율이 더 높은 걸로 보던가 아니면 하나만 출력하는 방식으로 선택 가능
                            # 두개를 주면 소프트 맥스
                            # 시그모이드로 할 경우에 하나로 설정, 그리고 임계치를 설정해줘야함. 보통 0.5. 절반으로 설정
])
model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x, labels, epochs =15)

In [30]:
print(f"최종훈련 정확도 : {history.history['acc'][-1]}")
preds = model.predict(x)
print(preds)
print(f'라벨: {labels}')

최종훈련 정확도 : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
[[0.47564846]
 [0.5005911 ]
 [0.4787427 ]
 [0.5030113 ]]
라벨: [0 1 0 1]


<span style="color: lightblue"> 시그모이드 vs 소프트 맥스를 언제 사용할까? </span>

<span style="font-size:11px;">

| 출력층 노드 수 | 활성화 함수 | 의미 | 동작 원리 | 클래스 결정 방식 | 사용 예시 |
|---------------|------------|------|------------|----------------|-----------|
| 1             | Sigmoid    | 0~1 범위 확률, 단일 출력으로 임계치 기준 판단 | 출력값 = 0~1 사이 실수<br>0.5 기준으로 판단<br>0.5 이상 → 클래스 1 (긍정)<br>0.5 미만 → 클래스 0 (부정) | 임계치(threshold, 보통 0.5) 기준 | 이진 분류 (Positive/Negative) |
| 2             | Softmax    | 각 클래스 확률, 출력값의 합 = 1 | 모든 출력의 합 = 1<br>각 값 = 해당 클래스 확률<br>예: [0.8, 0.2] → 긍정 80%, 부정 20% | argmax → 확률이 높은 클래스 선택 | 이진/다중 클래스 분류 |

---

예시 데이터로 실습

In [31]:
# nltk 데이터로드
import nltk
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\SAMSUNG\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [ ]:
# 재현성 시드 고정
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

In [34]:
# 데이터 로딩
ids = movie_reviews.fileids()
reviews = [movie_reviews.raw(id) for id in ids]
categories = [movie_reviews.categories(id)[0] for id in ids]

In [ ]:
# 하이퍼파라메터
    # 최대단어수
max_words = 10000
    # 문서길이
maxlen = 500
    # 임베딩차원
embed_dim = 64
    # batch_size
batch_size = 256
    # epoch
epochs = 15

# 토큰화 + 시퀀스변화 + 패딩

tokenizer = Tokenizer(num_words=50, oov_token = 'UNK') 
tokenizer.fit_on_texts(reviews)                       # 단어 사전 생성
x = tokenizer.texts_to_sequences(reviews)          # 단어 사전 기반으로 단어들을 숫자로 변경
# 패딩
x = pad_sequences(x, maxlen=maxlen, truncating = 'post')       # 길이가 짧으면 padding, 길이가 길면 truncating을 사용한다
# padding -> 짧은 시퀀스 뒤나 앞에 **특정 값(보통 0)**을 채워서 길이를 맞춤
# truncating -> 긴 시퀀스를 잘라서(maxlen으로) 길이를 맞춤
# 라벨 인코딩
label_map = {'pos':0, 'neg':1}
y = np.array([label_map.get(c) for c in categories])

# train/test 분할
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,stratify=y, random_state=42, test_size = 0.2)

# 모델구성
tf.keras.Sequential([
    tf.keras.layers.Embedding(max_words, out_put_dim = embed_dim, input_length = maxlen),
    tf.keras.layers.Conv1D(128, 3, activation='relu'),
    tf.keras.layer.MaxPool1D(), # 시퀀스의 구간별 특징 요약
    tf.keras.layers.Conv1D(256, 3, activation='relu'),
    tf.keras.layers.GlobalAvgPool1D()   # 전체 시퀀스의 평균 시퀀스 차원을 없애고 채널 축만 남김(채널에 대한 평균)
])
# 컴파일
# 콜백(선택)
# 학습
# 학습결과 시각화
# 테스트 평가
# 임의의 데이터로 예측

<span style="color: lightblue"> 모델구성 부분 설명

<span style="font-size:12px;">

| 레이어 | 역할 | 시각적 이해 |
|--------|------|------------|
| Embedding | 단어 → 벡터 | 각 단어를 “숫자 → 의미 벡터”로 변환. 예: `[1,2,3] → [[0.1,0.2],[0.5,0.1],[0.3,0.7]]` |
| Conv1D(128,3) | n-gram 특징 감지 | 3단어씩 묶어 슬라이딩하며 중요한 패턴 감지. 필터가 128개 → 128가지 특징을 동시에 학습 |
| MaxPool1D | 구간별 요약 | 슬라이딩된 출력에서 **가장 중요한 값만 추출**. 시퀀스 길이 감소 |
| Conv1D(256,3) | 더 복잡한 패턴 감지 | 이전 MaxPool 출력을 다시 스캔, 256개 특징 학습 |
| GlobalAvgPool1D | 시퀀스 요약 | 시퀀스 길이 전체 평균 → 고정 크기 벡터 (256차원) |